# Azure AI Foundry Agent Service - Single Agent MCP

Azure AI FoundryのMCP（Model Context Protocol）ツールを使用して外部サービスと連携するエージェントを構築するノートブックです。

## 概要

このノートブックでは、MCPツールを活用してMicrosoft Learn公式ドキュメントを検索できるエージェントを作成します。MCPプロトコルにより、エージェントが外部のAPIやサービスと安全かつ標準化された方法で連携する方法を学習します。

### 主な機能
- **MCP Tool**: Model Context Protocolによる外部サービス連携
- **ツール承認機能**: 自動承認と手動承認の両方をサポート

### MCPの利点
- **標準化**: 統一されたプロトコルによる外部連携
- **セキュリティ**: 承認プロセスや Entra ID 認証による安全な外部アクセス
- **拡張性**: 様々な外部サービスとの連携が可能
- **透明性**: ツール実行の詳細な記録と確認

このエージェントは、企業の内部システム連携、外部API統合、サードパーティサービス連携などの実用的なアプリケーションに応用できます。

# ライブラリのインポート

必要なPythonライブラリとAzure AI Foundry SDKをインポートします。

In [5]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)

# 環境変数の取得

Azure AI Foundryへの接続に必要な設定情報を環境変数から取得します。

- `PROJECT_ENDPOINT`: Azure AI Foundryプロジェクトのエンドポイント
- `AZURE_DEPLOYMENT_NAME`: 使用するAIモデルのデプロイメント名

これらの値は事前に.envファイルに設定されている必要があります。

In [6]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

Azure AI Foundryプロジェクトへの接続とエージェント管理用のクライアントを初期化します。

- `AIProjectClient`: Azure AI Foundryプロジェクトとの接続を管理
- `DefaultAzureCredential`: Azure認証（Azure CLI、Managed Identity等を自動検出）
- `agents_client`: エージェントの作成・管理・実行を行うクライアント

In [7]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

エージェントの実行結果を表示するためのヘルパー関数を定義します。

`agent_run_outputs`関数は以下の機能を提供します：
- スレッド内のメッセージ一覧を取得・表示
- 画像コンテンツがある場合は保存・表示

In [8]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

# ツールの定義

MCP（Model Context Protocol）ツールを設定し、外部サービスとの連携を準備します。

## MCPツールとは
Model Context Protocolは、言語モデルが外部ツール、メモリ、コンテキストと安全で構造化された状態を保持する方法で相互作用するための公開プロトコルです。

### MCPのアーキテクチャ
1. **Hosts**: MCPクライアントを使用してMCPサーバーからデータを接続・消費するアプリケーション
2. **Clients**: MCPホストのコンポーネントで、接続を管理しMCPサーバーからデータを取得
3. **Servers**: データリソース、アクション実行ツール、相互作用を導くプロンプトを提供するプログラム

### MCPツール設定項目
- **server_label**: MCPサーバーの一意識別子
- **server_url**: APIのエンドポイント
- **allowed_tools**: 利用可能なツールのリスト（オプション）
- **headers**: 認証やその他の目的でMCPサーバーに送信するHTTPヘッダー（オプション）

### 承認モード（require_approval）の詳細設定
1. **always**: すべてのツール呼び出しで承認が必要（デフォルト）
2. **never**: 承認不要
3. **{"never":["tool_name1", "tool_name2"]}**: 指定されたツールは承認不要
4. **{"always":["tool_name1", "tool_name2"]}**: 指定されたツールは承認必要

### セキュリティと認証
- **Entra ID統合**: Azure Identity ライブラリを通じたベストプラクティス認証
- **透明性**: 外部MCPサーバーとの共有データの明示的な承認プロセス
- **監査ログ**: リモートMCPサーバーとの共有データのロギング推奨

In [9]:
mcp_tool = McpTool(
    server_label="MicrosoftDocs",
    server_url="https://learn.microsoft.com/api/mcp",
)

# MCP ツール実行時の承認モードを never に設定
mcp_tool.set_approval_mode("never")

print(mcp_tool.resources)

{'mcp': [{'server_label': 'MicrosoftDocs', 'headers': {}, 'require_approval': 'never'}]}


# エージェントの作成

Foundry エージェントを作成します。

In [10]:
mcp_agent = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="mcp_agent",
    instructions=(
        "あなたは、MCPツールを使用してユーザーを支援できる有用なエージェントです。"
        "利用可能なMCPツールを使用して、質問に答えてタスクを実行します。"    
    ),
    description=(
        "あなたは Microsoft の公式ドキュメントを検索できる MCP ツールを持っているエージェントです。"
    ),
    tools=mcp_tool.definitions,
    tool_resources=mcp_tool.resources,
)
print(f"Created Agent. AGENT_ID: {mcp_agent.id}")


Created Agent. AGENT_ID: asst_LYbl3utUTzFdl9RV7iQU5LjT


In [11]:
agent_dict = mcp_agent.as_dict()
print(json.dumps(agent_dict, indent=2, ensure_ascii=False))

{
  "id": "asst_LYbl3utUTzFdl9RV7iQU5LjT",
  "object": "assistant",
  "created_at": 1753694766,
  "name": "mcp_agent",
  "description": "あなたは Microsoft の公式ドキュメントを検索できる MCP ツールを持っているエージェントです。",
  "model": "gpt-4.1",
  "instructions": "あなたは、MCPツールを使用してユーザーを支援できる有用なエージェントです。利用可能なMCPツールを使用して、質問に答えてタスクを実行します。",
  "tools": [
    {
      "type": "mcp",
      "server_label": "MicrosoftDocs",
      "server_url": "https://learn.microsoft.com/api/mcp",
      "allowed_tools": null
    }
  ],
  "top_p": 1.0,
  "temperature": 1.0,
  "tool_resources": {},
  "metadata": {},
  "response_format": "auto"
}


# スレッドの作成

エージェントとの対話を管理するスレッドを作成します。

In [12]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_fFFfnqTX9fofwwzWfaKjleVy


# ユーザーメッセージの追加

スレッドにユーザーからの質問やリクエストを追加します。

In [13]:
# メッセージの追加
user_message = "Azure AI Foundry Agent Service の最新情報を教えてください。"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_uSKWbbT7dkxw3zLnMbWbjkci


# Run の実行

エージェントにタスクを実行させ、結果を取得します。

## MCP ツール自動承認で実行

承認モードを"never"に設定したMCPツールを使用して、自動承認でエージェントを実行します。

In [14]:
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=mcp_agent.id,
    tool_resources=mcp_tool.resources # MCP ツールのリソース情報を渡す（承認モード設定やリクエストヘッダーの設定はここで定義される）
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: Azure AI Foundry Agent Service の最新情報を教えてください。
ASSISTANT: Azure AI Foundry Agent Serviceの最新情報（2025年6月時点）は以下の通りです：

### 2025年6月の主なアップデート

- **Deep Research Tool**
  - Azure OpenAIの `o3-deep-research` モデルとBing検索を組み合わせたマルチステップリサーチツールが追加。
  - 詳細はこちら: Deep Research tool

- **Model Context Protocol (MCP) Tool**
  - 遠隔のMCPサーバー上にホストされるツールにエージェントを接続し、機能拡張が可能に。
  - 詳細はこちら: MCP tool

### 2025年5月の主なアップデート

- **一般提供（GA）開始**
  - Azure AI Foundry Agent Serviceが一般提供に移行。

- **新機能**
  - Visual Studio Code拡張機能によるエージェントのデプロイ・管理が可能に。
  - 「Connected agents」で、主エージェントと連携可能なタスク特化エージェントを作成可能。
  - エージェント実行の入力・出力のトレース（実行経路解析）が可能に。
  - Azure Logic Apps からイベントトリガーでエージェントを自動起動。
  - 新ツール:
    - Bing Custom Search
    - Morningstarデータ連携

### 2025年4月 〜 2024年12月の主なアップデート

- **Azure Monitor連携による運用観測の強化**
- **Cosmos DBのBYO（Bring Your Own）スレッドストレージ対応**
- **Microsoft Fabricとの連携ツール**
- **Azure AI Foundryポータルでのノーコード運用**

### 追加情報

- 複数のAIモデル（Llama 3.1, Mistral, Cohere等）への対応が拡大。
- セキュリティや監査機能（トレーシング、Application Insights連携

## MCP ツール手動承認で実行

MCPツールの実行に対して手動で承認を行うプロセスを実装します。

### 手動承認の実装流れ
1. **Run作成**: `create()`でRunを作成（`create_and_process()`は使用しない）
2. **状態監視**: `requires_action`状態になるまでポーリング
3. **承認判断**: `SubmitToolApprovalAction`が要求されたら承認処理
4. **ToolApproval作成**: 各ツール呼び出しに対して承認オブジェクトを作成
5. **承認送信**: `submit_tool_outputs()`で承認結果を送信

### 手動承認の利点
- **セキュリティ**: 外部API呼び出しの詳細確認と制御
- **透明性**: ツール実行内容の事前確認
- **コスト管理**: 不要な外部API呼び出しの防止
- **コンプライアンス**: 企業ポリシーに応じた承認プロセス

### 承認オブジェクトの設定
- **tool_call_id**: 承認対象のツール呼び出しID
- **approve**: True（承認）またはFalse（拒否）
- **headers**: MCPツールのヘッダー情報

In [15]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

# メッセージの追加
user_message = "Azure AI Foundry Agent Service の最新情報を教えてください。"
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Added Message. MESSAGE_ID: {message.id}")

# Run の作成と実行
run = agents_client.runs.create(
    thread_id=thread.id,
    agent_id=mcp_agent.id
)

# Run の状態をポーリングして確認
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True, # 承認する場合は True、拒否する場合は False
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            # ツール承認を送信
            agents_client.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")

Created Thread. THREAD_ID: thread_208rapESXmSsF5CYdYqqobv3
Added Message. MESSAGE_ID: msg_ubgqlwDyoXncCIN2rASZ0Tpb
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Approving tool call: {'id': 'call_WxmGfLs2hWQuwwSrF6vKqPMR', 'type': 'mcp', 'arguments': '{"question":"Azure AI Foundry Agent Service ã\x81®æ\x9c\x80æ\x96°æ\x83\x85å\xa0±"}', 'name': 'microsoft_docs_search', 'server_label': 'MicrosoftDocs'}
tool_approvals: [{'tool_call_id': 'call_WxmGfLs2hWQuwwSrF6vKqPMR', 'approve': True, 'headers': {}}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PRO

In [16]:
agent_run_outputs(thread.id, agents_client)

USER: Azure AI Foundry Agent Service の最新情報を教えてください。
ASSISTANT: Azure AI Foundry Agent Serviceの最新情報（2025年6月時点）は以下のとおりです。

### 最新リリース・重要なアップデート

#### 2025年6月
- **Deep Researchツール**  
  Azure OpenAIのo3-deep-researchモデルとBing Searchを活用したマルチステップリサーチツールが利用可能になりました。  
  [Deep Researchツールの詳細](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research)

- **Model Context Protocol (MCP) ツール**  
  エージェントからリモートのMCPサーバー上にホストされたツールへ接続可能になり、エージェントの機能拡張ができるようになりました。  
  [MCPツールの詳細](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)

#### 2025年5月
- **一般提供開始（GA）**
    - Azure AI Foundry Agent Serviceが一般提供となりました。
    - Visual Studio Code拡張でエージェントのデプロイ・管理が可能。
    - 「Connected Agents」により、複数のエージェントを1つのシステム内で連携可能に。
    - エージェントのトレース機能（入出力経路の可視化）が追加。
    - Azure Logic Appsによるトリガー実行（特定イベント発生時に自動でエージェントを呼び出し）。
    - Bing Custom Search、Morningstarなど新ツールへの対応。

#### 2025年4月以前
- Azure Monitorとの統合で指標（実行数、ファイル数など）が参照可能に。
- Azure Cosmos DBを利用したBYO（Br

# トレースの確認

[Azure AI Foundry Portal](https://ai.azure.com/?cid=learnDocs) でエージェントの実行トレースを確認してみましょう。

# Agent ID を .env ファイルに保存

作成したFile SearchエージェントのIDを永続化し、他のノートブックで再利用できるようにします。

In [ ]:
# 変数の定義
agent_env_key = "FOUNDRY_MCP_AGENT_ID"
agent_env_value = mcp_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')


.envファイルに FOUNDRY_MCP_AGENT_ID="asst_LYbl3utUTzFdl9RV7iQU5LjT" を追記しました。


True